In [163]:
import argparse
import re

remove_c_tags = lambda x: x.replace('<c>', '').replace('</c>', '')

def cue_time_to_seconds(cue_time):
    h, m, seconds = cue_time[1:-1].split(':')
    s, ms = seconds.split('.')
    return int(h)*60*60 + int(m)*60 + int(s) + float(ms)/1000
    
    
master_cue_list = []

video_code = 'HqzxtaTAfm4'
caption_filepath = './raw-captions/HqzxtaTAfm4.en.vtt'

with open(caption_filepath) as f:
    saw_line_timing = False
    line_start = None
    line_end = None
    
    for line in f.readlines():
        if '-->' in line:
            if saw_line_timing:
                master_cue_list += [line_start, line_end]
                
            saw_line_timing = True
            start = f'<{line.split()[0]}>'
            end = f'<{line.split()[2]}>'
            master_cue_list += [start]
            line_start = start
            line_end = end

        if '<c>' in line:
            saw_line_timing = False
            cue_line_list = remove_c_tags(line).replace('<', ' <').split()
            master_cue_list += cue_line_list
            master_cue_list += [line_end]

            
for i, entry in enumerate(master_cue_list):
    if '<' in entry:        
        seconds = cue_time_to_seconds(entry)
        master_cue_list[i] = seconds
        
print(master_cue_list)
        

[8.33, 'okay', 9.33, 'look', 9.57, 'I', 9.69, 'just', 9.719, 'conferred', 10.32, 'with', 10.349, 'the', 10.71, 'sin', 11.0, 11.0, 11.0, 11.01, 11.01, 'counter', 11.37, 'and', 11.639, "it's", 12.03, 'telling', 12.33, 'me', 12.36, 'that', 12.509, 'it', 12.89, 12.89, 12.89, 12.9, 12.9, "doesn't", 13.17, 'want', 13.23, 'any', 13.559, 'part', 13.889, "I'm", 14.099, 'sending', 14.49, 'a', 14.57, 14.57, 14.57, 14.58, 14.58, 'Stanley', 14.969, 'Kubrick', 15.24, 'movie', 15.48, 'so', 15.78, 'the', 15.96, 'sin', 16.139, 'counter', 16.43, 16.43, 16.43, 16.44, 16.44, 'is', 16.56, 'gonna', 16.71, 'leave', 16.89, 'we', 17.46, 'scoured', 17.789, 'the', 17.85, 'internet', 18.109, 18.109, 18.109, 18.119, 18.119, 'and', 18.27, 'could', 18.51, 'only', 18.66, 'find', 18.99, 'one', 19.109, 'being', 19.529, 'willing', 19.89, 'to', 19.939, 19.939, 19.939, 19.949, 19.949, 'do', 20.07, 'this', 20.22, 'for', 20.43, 'us', 20.46, 'and', 20.76, 'apparently', 21.42, "it's", 21.57, 'the', 21.679, 21.679, 21.679, 21.

In [176]:
import csv
import os

def write_to_tsv(row, header, filepath):
    mode = 'w'
    if os.path.exists(filepath):
        mode = 'a'
    
    with open(filepath, mode) as f:
        writer = csv.writer(f, delimiter='\t', lineterminator='\n')
        if mode == 'w':
            writer.writerow(header)
        writer.writerow(row)

In [180]:
vocab_directory = './vocabulary'

for i, entry in enumerate(master_cue_list):
    if type(entry) == str:
        word = entry
        start_time = master_cue_list[i-1]
        end_time = master_cue_list[i+1]

        vocab_subdirectory = word[0].upper()
        safe_word = ''.join([c for c in word.lower() if c.isalpha() or c.isdigit() or c==' ']).rstrip()

        filename = f'{safe_word}.tsv'
        dir_path = f'{vocab_directory}/{vocab_subdirectory}'
        
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

        write_to_tsv([video_code, start_time, end_time], ['video_code', 'start_time', 'end_time'], f'{dir_path}/{filename}')
        
        
        

        
        